## Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random
import pandas as pd
import numpy as np
import os
# import zipfile
import cv2
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torchvision.models as models

from sklearn import metrics
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
%cd /content
os.makedirs('./project/input/train_imgs', exist_ok=True)
os.makedirs('./project/input/train_masks', exist_ok=True)
os.makedirs('./project/input/test_imgs', exist_ok=True)

/content


In [ ]:
%cd ./project/input

/content/project/input


In [ ]:
import os
 
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [ ]:
"""# input data 압 축 풀 기 한 번 만 ! 
import zipfile

from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/project/input/'

output_unzip = zipfile.ZipFile("/content/drive/MyDrive/project/BreastCancer/input/open.zip", "r")  # "r": read 모드
output_unzip.extractall(file_path)
output_unzip.close()"""

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
"""import zipfile

augu_file_path = file_path+'/augu/aug_E2/'
createFolder(augu_file_path)

output_unzip = zipfile.ZipFile("/content/drive/MyDrive/BreastCancer/input/augu/aug_E2.zip", "r")  # "r": read 모드
output_unzip.extractall(augu_file_path)
output_unzip.close()

print(len(os.listdir(augu_file_path)))"""

1000


In [ ]:
file_path = '/content/project/input/'

augu_file_path = file_path+'/augu/aug_E2/'
createFolder(augu_file_path)
print(len(os.listdir(augu_file_path)))

In [ ]:
train_df = pd.read_csv('/content/project/input/train.csv')
test_df = pd.read_csv('/content/project/input/test.csv')
len(train_df),len(test_df)

In [ ]:
df_Aug = pd.DataFrame()

for file_name in tqdm(os.listdir(augu_file_path)):
  if len(file_name) == 1:
    continue
  if file_name == '.ipynb_checkpoints':
    continue
  img_ID = file_name.replace('.png', '').split('__')[0]
  df = train_df[train_df['ID']==img_ID]
  df['img_path']= './augu/aug_E2/'+file_name
  df_Aug =pd.concat([df_Aug,df])  
  #shutil.move(f'/content/data/content/mmclassification/data/train_imgs/{i}', f'./data/dataset/training_set/training_set/{k}')

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
df_Aug

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,...,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category
298,BC_01_1419,./augu/aug_E2/BC_01_1419__coll00.png,-,63,2017-01-13,1,2,2,25.0,3.0,...,8.0,1.0,7.0,20.78,0.0,2.0,0.0,1.08,NaN,1
817,BC_01_2938,./augu/aug_E2/BC_01_2938__coll00.png,-,55,2008-02-01,1,1,1,10.0,NaN,...,NaN,1.0,NaN,NaN,1.0,3.0,NaN,NaN,NaN,0
803,BC_01_2922,./augu/aug_E2/BC_01_2922__coll00.png,-,35,2008-03-11,1,2,1,21.0,NaN,...,NaN,1.0,NaN,NaN,1.0,2.0,NaN,NaN,NaN,0
578,BC_01_2599,./augu/aug_E2/BC_01_2599__coll00.png,-,47,2012-11-20,1,2,1,4.0,2.0,...,5.0,0.0,NaN,30.00,1.0,3.0,NaN,NaN,NaN,0
711,BC_01_2789,./augu/aug_E2/BC_01_2789__coll00.png,-,68,2011-04-28,1,2,1,10.0,2.0,...,8.0,1.0,5.0,10.00,1.0,2.0,1.0,2.00,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,BC_01_0003,./augu/aug_E2/BC_01_0003__coll00.png,-,37,2015-10-29,1,2,1,NaN,2.0,...,7.0,1.0,4.0,7.00,0.0,1.0,NaN,NaN,0.0,0
594,BC_01_2640,./augu/aug_E2/BC_01_2640__coll00.png,-,40,2012-06-15,2,2,1,15.0,3.0,...,7.0,1.0,7.0,3.00,0.0,0.0,NaN,NaN,NaN,0
839,BC_01_2966,./augu/aug_E2/BC_01_2966__coll00.png,-,54,2007-11-09,1,2,1,15.0,NaN,...,NaN,1.0,NaN,NaN,0.0,1.0,NaN,NaN,NaN,1
570,BC_01_2587,./augu/aug_E2/BC_01_2587__coll00.png,-,59,2012-12-14,4,2,1,11.0,1.0,...,7.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0


In [ ]:
train_df = pd.concat([df_Aug,train_df])
train_df

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,...,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category
298,BC_01_1419,./augu/aug_E2/BC_01_1419__coll00.png,-,63,2017-01-13,1,2,2,25.0,3.0,...,8.0,1.0,7.0,20.78,0.0,2.0,0.0,1.08,NaN,1
817,BC_01_2938,./augu/aug_E2/BC_01_2938__coll00.png,-,55,2008-02-01,1,1,1,10.0,NaN,...,NaN,1.0,NaN,NaN,1.0,3.0,NaN,NaN,NaN,0
803,BC_01_2922,./augu/aug_E2/BC_01_2922__coll00.png,-,35,2008-03-11,1,2,1,21.0,NaN,...,NaN,1.0,NaN,NaN,1.0,2.0,NaN,NaN,NaN,0
578,BC_01_2599,./augu/aug_E2/BC_01_2599__coll00.png,-,47,2012-11-20,1,2,1,4.0,2.0,...,5.0,0.0,NaN,30.00,1.0,3.0,NaN,NaN,NaN,0
711,BC_01_2789,./augu/aug_E2/BC_01_2789__coll00.png,-,68,2011-04-28,1,2,1,10.0,2.0,...,8.0,1.0,5.0,10.00,1.0,2.0,1.0,2.00,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,BC_01_3464,./train_imgs/BC_01_3464.png,-,65,2006-12-22,1,2,1,25.0,1.0,...,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1
996,BC_01_3482,./train_imgs/BC_01_3482.png,-,48,2006-11-17,1,1,1,7.0,1.0,...,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0
997,BC_01_3485,./train_imgs/BC_01_3485.png,-,64,2006-11-10,1,2,1,15.0,1.0,...,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1
998,BC_01_3502,./train_imgs/BC_01_3502.png,-,50,2006-09-22,1,1,1,7.0,1.0,...,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0


In [ ]:
device = torch.device('cuda') \
if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

## Hyperparameter Setting

In [ ]:
# Cfg, 시드 정하기!!!!!!!
EPOCHS = 30
MODEL = 'resnext50_32x4d'
VERSION = 'E2'

CFG = {
    'IMG_SIZE': 512,
    'EPOCHS': EPOCHS,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':16,
    'SEED':41
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [ ]:
train_df['암의 장경'] = train_df['암의 장경'].fillna(train_df['암의 장경'].median())
train_df = train_df.fillna(0)

test_df['암의 장경'] = test_df['암의 장경'].fillna(train_df['암의 장경'].median())
test_df = test_df.fillna(0)

In [ ]:
train_labels = train_df['N_category']
train_df = train_df.drop(columns=['N_category'])

def get_values(value):
    return value.values.reshape(-1, 1)

numeric_cols = ['나이', '암의 장경', 'ER_Allred_score', 'PR_Allred_score', 'KI-67_LI_percent', 'HER2_SISH_ratio']
ignore_cols = ['ID', 'img_path', 'mask_path', '수술연월일', 'N_category']

for col in train_df.columns:
    if col in ignore_cols:
        continue
    if col in numeric_cols:
        scaler = StandardScaler()
        train_df[col] = scaler.fit_transform(get_values(train_df[col])) # 범위의 스케일을 조절
        test_df[col] = scaler.transform(get_values(test_df[col]))
    else:
        le = LabelEncoder()
        train_df[col] = le.fit_transform(get_values(train_df[col])) # 수치형 라벨화
        test_df[col] = le.transform(get_values(test_df[col]))

## CustomDataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, medical_df, labels, transforms=None):
        self.medical_df = medical_df
        self.transforms = transforms
        self.labels = labels
        
    def __getitem__(self, index):
        img_path = self.medical_df['img_path'].iloc[index]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
                
        if self.labels is not None:
            tabular = torch.Tensor(self.medical_df.drop(columns=['ID', 'img_path', 'mask_path', '수술연월일']).iloc[index])
            label = self.labels[index]
            return image, tabular, label
        else:
            tabular = torch.Tensor(self.medical_df.drop(columns=['ID', 'img_path', '수술연월일']).iloc[index])
            return image, tabular

    def __len__(self):
        return len(self.medical_df)

In [ ]:
train_transforms = A.Compose([
                            A.HorizontalFlip(),
                            A.VerticalFlip(),
                            A.Rotate(limit=90, border_mode=cv2.BORDER_CONSTANT,p=0.3),
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transforms = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])


# mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)

## Model Architecture

In [ ]:
##gdrive path 설정
GPATH = '/content/drive/MyDrive/BreastCancer/baseline/kFold/'
print(GPATH)
createFolder(GPATH)

/content/drive/MyDrive/BreastCancer/baseline/kFold/


In [ ]:
class ImgFeatureExtractor(nn.Module): # 이미지 피처값
    def __init__(self):
        super(ImgFeatureExtractor, self).__init__()
        self.backbone = models.resnext50_32x4d(pretrained=True) # backbone efficientnet_b0
        self.embedding = nn.Linear(1000,512)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.embedding(x)
        return x

In [ ]:
class TabularFeatureExtractor(nn.Module): # 정형화 데이터 피처값 추출
    def __init__(self):
        super(TabularFeatureExtractor, self).__init__()
        self.embedding = nn.Sequential(
            nn.Linear(in_features=23, out_features=128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Linear(in_features=512, out_features=512)
        )
        
    def forward(self, x):
        x = self.embedding(x)
        return x

In [ ]:
class ClassificationModel(nn.Module): # 멀티모달
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.img_feature_extractor = ImgFeatureExtractor()
        self.tabular_feature_extractor = TabularFeatureExtractor()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=1024, out_features=1),
            nn.Sigmoid()
        )
        
    def forward(self, img, tabular):
        img_feature = self.img_feature_extractor(img)
        tabular_feature = self.tabular_feature_extractor(tabular)
        feature = torch.cat([img_feature, tabular_feature], dim=-1)
        output = self.classifier(feature)
        return output

## Train

In [ ]:
from sklearn.model_selection import StratifiedKFold

kfold_split_num = 5

str_kf = StratifiedKFold(n_splits = kfold_split_num, shuffle = True, random_state = CFG['SEED'])

def train(model, optimizer, df, df_labels, scheduler, device):
    dic_log={
        'epoch':[],
        'train_loss':[],
        'val_loss':[],
        'val_score':[]
    }
    model.to(device)
    criterion = nn.BCEWithLogitsLoss().to(device)
        
    best_score = 0
    best_model = None
    
    for epoch in range(CFG['EPOCHS'] // kfold_split_num):
        model.train()
        train_loss = []

        temp = 1

        for train_index, test_index in str_kf.split(df, df_labels):
            X_train, X_test = df.iloc[train_index], df.iloc[test_index]
            y_train, y_test = df_labels.iloc[train_index], df_labels.iloc[test_index]

            train_dataset = CustomDataset(X_train, y_train.values, train_transforms)
            val_dataset = CustomDataset(X_test, y_test.values, test_transforms)

            train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
            val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

            for img, tabular, label in tqdm(iter(train_loader)):
                img = img.float().to(device)
                tabular = tabular.float().to(device)
                label = label.float().to(device)
                
                optimizer.zero_grad()
                
                model_pred = model(img, tabular)
                
                loss = criterion(model_pred, label.reshape(-1,1))
                
                loss.backward()
                optimizer.step()
                
                train_loss.append(loss.item())
            
            val_loss, val_score = validation(model, criterion, val_loader, device)
            juist_epoch = epoch * kfold_split_num + temp
            print(f'Epoch [{juist_epoch}], Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}] Val Score : [{val_score:.5f}]')
            ##### 에폭마다 log 저장하기!!!
            dic_log['epoch'].append(epoch)
            dic_log['train_loss'].append(np.mean(train_loss))
            dic_log['val_loss'].append(val_loss)
            dic_log['val_score'].append(val_score)

            if scheduler is not None:
                scheduler.step(val_score)
            
            if best_score < val_score:
                best_score = val_score
                best_model = model
                # best 뽑기
                test_dataset = CustomDataset(test_df, None, test_transforms)
                test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
                torch.save(model, GPATH+f'{MODEL}/{VERSION}/ckp/best_{MODEL}_{epoch}_{temp}.pt') # 각 epoch가 끝날 때마다 모델 상태 저장
                predictions,preds = inference(best_model, test_loader, device)
                print('########################################################')
                submit = pd.read_csv('./sample_submission.csv')
                submit['N_category'] = preds
                submit['predictions'] = predictions
                submit.to_csv(GPATH+f'{MODEL}/{VERSION}/submit/best_submit_{MODEL}_{EPOCHS}_{epoch}_{temp}.csv', index=False)
                print(f'Epoch [{juist_epoch}]...Saved')
    
            if CFG['EPOCHS'] == juist_epoch:
                # 로그 저장하기!
                df_log = pd.DataFrame(dic_log)
                df_log.to_csv(GPATH+ f'{MODEL}/{VERSION}/log_{MODEL}_{VERSION}_{temp}.csv',index=False)
                return best_model
            
            
            torch.save(model, GPATH+f'{MODEL}/{VERSION}/ckp/checkpoint_{MODEL}_{epoch}_{temp}.pt') # 각 epoch가 끝날 때마다 모델 상태 저장
            print('########################################################')
            ##### 에폭마다 submission 뽑기!
            submit = pd.read_csv('./sample_submission.csv')
            predictions,preds = inference(model, test_loader, device)
            submit['N_category'] = preds
            submit['predictions'] = predictions
            submit.to_csv(GPATH+ f'{MODEL}/{VERSION}/submit/submit_{MODEL}_{epoch}_{temp}.csv', index=False)

            temp += 1


####################################################################################################################
####################################################################################################################
####################################################################################################################

def validation(model, criterion, val_loader, device):
    model.eval()
    pred_labels = []
    true_labels = []
    val_loss = []
    threshold = 0.4
    with torch.no_grad():
        for img, tabular, label in tqdm(iter(val_loader)):
            true_labels += label.tolist()
            
            img = img.float().to(device)
            tabular = tabular.float().to(device)
            label = label.float().to(device)
            
            model_pred = model(img, tabular)
            
            loss = criterion(model_pred, label.reshape(-1,1))
            
            val_loss.append(loss.item())
            
            model_pred = model_pred.squeeze(1).to('cpu')  
            pred_labels += model_pred.tolist()
    
    pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
    val_score = metrics.f1_score(y_true=true_labels, y_pred=pred_labels, average='macro')
    return np.mean(val_loss), val_score

####################################################################################################################
####################################################################################################################
####################################################################################################################


def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    threshold = 0.4
    
    with torch.no_grad():
        for img, tabular in tqdm(iter(test_loader)):
            img = img.float().to(device)
            tabular = tabular.float().to(device)
            
            model_pred = model(img, tabular)
            
            model_pred = model_pred.squeeze(1).to('cpu')
            
            preds += model_pred.tolist()
    
    predictions=np.array(preds)
    preds = np.where(np.array(preds) > threshold, 1, 0)
    
    return predictions,preds


## Run!!

In [ ]:
print(CFG)
print(GPATH,MODEL,VERSION)

{'IMG_SIZE': 512, 'EPOCHS': 30, 'LEARNING_RATE': 0.0001, 'BATCH_SIZE': 16, 'SEED': 41}
/content/drive/MyDrive/BreastCancer/baseline/kFold/ resnext50_32x4d E2


In [ ]:
createFolder(GPATH+'/'+MODEL+'/'+str(VERSION))
createFolder(GPATH+'/'+MODEL+'/'+str(VERSION)+'/ckp/')
createFolder(GPATH+'/'+MODEL+'/'+str(VERSION)+'/submit/')
print(GPATH+'/'+MODEL+'/'+str(VERSION))


/content/drive/MyDrive/BreastCancer/baseline/kFold//resnext50_32x4d/E2


In [ ]:
device = torch.device("cuda")
model = nn.DataParallel(ClassificationModel())

# 학습 이어 하기
#PATH = '/content/drive/MyDrive/project/BreastCancer/baseline/resnext50_32x4d/1/ckp/best_resnext50_32x4d_9.pt'
#model = torch.load(PATH)

model.eval()
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1, threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_df, train_labels, scheduler, device) # val_loader

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.63976] Val Loss : [0.61434] Val Score : [0.72335]


  0%|          | 0/16 [00:00<?, ?it/s]

########################################################
Epoch [1]...Saved
########################################################


...

########################################################
Epoch [29]...Saved
########################################################


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch [30], Train Loss : [0.54597] Val Loss : [0.54316] Val Score : [0.90240]


## Inference

In [ ]:
test_dataset = CustomDataset(test_df, None, test_transforms)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], num_workers=0, shuffle=False)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    threshold = 0.4
    
    with torch.no_grad():
        for img, tabular in tqdm(iter(test_loader)):
            img = img.float().to(device)
            tabular = tabular.float().to(device)
            
            model_pred = model(img, tabular)
            
            model_pred = model_pred.squeeze(1).to('cpu')
           
            preds += model_pred.tolist()
    score = preds
    preds = np.where(np.array(preds) > threshold, 1, 0)
    
    return preds, score

In [ ]:
preds, score = inference(infer_model, test_loader, device)

  0%|          | 0/16 [00:00<?, ?it/s]

## Submission

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['N_category'] = preds
submit['score'] = score
submit

,ID,N_category,score
0,BC_01_0011,1,0.999884
1,BC_01_0220,0,0.000372
2,BC_01_0233,1,0.437494
3,BC_01_0258,0,0.000091
4,BC_01_0260,1,1.000000
...,...,...,...
245,BC_01_3328,1,0.999819
246,BC_01_3404,0,0.228130
247,BC_01_3418,0,0.000086
248,BC_01_3438,0,0.000011


In [ ]:
submit.to_csv(GPATH+ f'{MODEL}/{VERSION}/submit/submit_{MODEL}_{EPOCHS}_{VERSION}.csv', index=False)